In [25]:
# !pip install yfinance mplfinance

In [26]:
import pandas as pd, yfinance as yf
from pathlib import Path
import mplfinance as mpf


df = yf.download("QQQ",start="2000-01-01",end="2026-01-01", group_by="column", interval="1d", auto_adjust=True)
df.columns.name = None
df.columns = df.columns.get_level_values(0)
df = df[["Open", "High", "Low", "Close"]]
df.columns.name = None
df.index.name = "Date"
df = df.astype(float)
df.index = pd.to_datetime(df.index)

bull_dir = Path("charts/bullish")
bear_dir = Path("charts/bearish")

bull_dir.mkdir(parents=True, exist_ok=True)
bear_dir.mkdir(parents=True, exist_ok=True)

window = 30
future_days = 5
ret_threshold = 0.015


stride = 3

for i in range(0, len(df) - window - future_days, stride):

    chunk = df.iloc[i : i + window]
    future = df.iloc[i + window : i + window + future_days]
    prev_high = chunk["High"].max()
    prev_low  = chunk["Low"].min()
    future_high = future["High"].max()
    future_low  = future["Low"].min()
    future_return = (future["Close"].iloc[-1] / future["Open"].iloc[0]) - 1

    if future_return > ret_threshold and future_high > prev_high:
        out_dir = bull_dir
        tag = "bullish"
    elif future_return < -ret_threshold and future_low < prev_low:
        out_dir = bear_dir
        tag = "bearish"
    else:
        continue

    start = chunk.index[0].date()
    end   = chunk.index[-1].date()

    fname = out_dir / f"QQQ_{start}_to_{end}_{tag}.png"
    
    mpf.plot(chunk, type="candle", style="yahoo", figsize=(10, 5), savefig=fname, tight_layout=True, datetime_format="%m-%d")

print("Bullish:", len(list(bull_dir.glob("*.png"))))
print("Bearish:", len(list(bear_dir.glob("*.png"))))



[*********************100%***********************]  1 of 1 completed


Bullish: 362
Bearish: 206
